In [1]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import gensim
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix

In [2]:
import pandas as pd, numpy as np
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path

In [3]:
# Reading PIMMART data
pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
    pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)

# Reading Syndigo 259K data
synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
    synd_ALL[i] = synd_ALL[i].astype(np.float64)

# Stripping spaces from all columns
df_obj = synd_ALL.select_dtypes(['object'])
synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [4]:
syndigo_mapped = synd_ALL
pimmart = pim_gtin_mapped

In [5]:
syndigo_mapped.drop_duplicates('GTIN', inplace = True)

In [6]:
syndigo_mapped['ITEM_SUBCOM_text'] = \
(syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()

In [7]:
syndigo_mapped['Level 1'].value_counts()

Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
Not classified                                1018
Hardware                                       923
Arts & Crafts                                  803
Automotive                                     778
Appliances                     

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range= (1,2), max_features = 100000) # from 800000 to 100000

In [9]:
syndigo_mapped['Level 1'].value_counts()

Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
Not classified                                1018
Hardware                                       923
Arts & Crafts                                  803
Automotive                                     778
Appliances                     

## RUN UNTIL HERE only ONCE

In [10]:
level__1 = "Toys / Games / Hobbies"
filenamee = "TOYS_GAMES_HOBBIES"

#level__1 = "Tobacco Products"
#filenamee = "TOBACCO_PRODUCTS"

#level__1 = "Cleaning & Janitorial"
#filenamee = "CLEANING_JANITORIAL"

# level__1 = "Office Supplies"
# filenamee = "OFFICE_SUPPLIES"

# level__1 = "Beer / Wine / Spirits"
# filenamee = "BEER_WINE_SPIRITS"

# level__1 = "Livestock & Pet Supplies"
# filenamee = "LIVESTOCK_PET_SUPPLIES"

# level__1 = "Kitchen & Bathroom"
# filenamee = "KITCHEN_BATHROOM"

# level__1 = "Hospitality Supplies"
# filenamee = "HOSPITALITY_SUPPLIES"

# level__1 = "Home & Venue Decoration"
# filenamee = "HOME_VENUE_DECORATION"

# level__1 = "Heating / Ventilation / Air Conditioning"
# filenamee = "HEATING_VENTILATION_AIR_CONDITIONING"

# level__1 = "Gardening & Outdoors"
# filenamee = "GARDENING_OUTDOORS"

# level__1 = "Sports & Outdoor Recreation Equipment"
# filenamee = "SPORTS_OUTDOOR_RECREATION_EQUIPMENT"

syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)

Hobbies                           1
Nature / Science                 12
Remote & Radio Control           38
Sports Toys & Games              82
Role Playing & Pretend Play     164
Outdoor Toys & Games            191
Toy Vehicles                    258
Baby Toys & Games               372
Construction & Modeling         394
Games & Puzzles                 540
Toy Figures                    1139
Name: Level 2, dtype: int64

In [ ]:
level_1 = level__1
subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]
series = syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)
if series.tolist():
    subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series==1].index.tolist())].index)

vect_subset = TfidfVectorizer(ngram_range = (1,2), max_features = 100000)# from 1000000 to 100000
X_subset = vect_subset.fit_transform(subset_df.ITEM_SUBCOM_text)
level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
y_subset  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
A_train, A_test, B_train, B_test = train_test_split(subset_df.GTIN.tolist(), subset_df['Level 2'].tolist(), test_size=0.1, stratify=y_subset, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size= 0.1, stratify=y_subset, random_state=42)
lr_tf = LogisticRegressionCV(cv = 5, multi_class = 'multinomial', solver = 'saga', n_jobs=-1, max_iter=10000000)

print(f"Training - \"{level_1}\"")
lr_tf.fit(X_train, y_train)
preds = lr_tf.predict(X_test)
preds_lrtf = preds
probs = lr_tf.predict_proba(X_test)
preds_train = lr_tf.predict(X_train)
probs_train = lr_tf.predict_proba(X_train)
print(f"Done Training - \"{level_1}\"")

Training - "Toys / Games / Hobbies"


In [ ]:
level_num = []
testProobs = []
number = 1

print(len(preds))
for i in range(len(preds)):
    for j in range(number):
        try:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][j])[0]))
        except TypeError:
            level_num.append(int(np.where(probs[i]==probs[i][np.argsort(probs[i])][::-1][:3][2])[0][0]))
            print("problem occurred with: ", i, j, flush=True)
    testProobs.append(probs[i][np.argsort(probs[i])][::-1][:1])

test_new_proobs = []
for i in range(len(testProobs)):
    test_new_proobs.append(testProobs[i].tolist())
test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]

In [ ]:
level_num = []
trainProobs = []

print(len(preds_train))
for i in range(len(preds_train)):
    for j in range(number):
        try:
            level_num.append(int(np.where(probs_train[i]==probs_train[i][np.argsort(probs_train[i])][::-1][:3][j])[0]))
        except TypeError:
            level_num.append(int(np.where(probs_train[i]==probs_train[i][np.argsort(probs_train[i])][::-1][:3][2])[0][0]))
            print("problem occurred with: ", i, j, flush=True)
    trainProobs.append(probs_train[i][np.argsort(probs_train[i])][::-1][:1]) 

train_new_proobs = []
for i in range(len(trainProobs)):
    train_new_proobs.append(trainProobs[i].tolist())
train_new_proobs = [element for sublist in list(train_new_proobs) for element in sublist]

In [ ]:
testLevels = []
for j in y_test:
    testLevels.append([i for i in level2_id_map if level2_id_map[i]==j][0])

testLevelss = []
for j in preds:
    testLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])

trainLevelss = []
for j in preds_train:
    trainLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])

In [ ]:
print("---------------------------------------------\FINAL COUNTS:\n---------------------------------------------")
print("1)", len(A_test + A_train))
print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
print("3)", len(B_test + B_train))
print("4)", len(testLevels + B_train))
print("5)", len(testLevelss + trainLevelss))
print("6)", int(len(test_new_proobs + train_new_proobs)))

In [ ]:
data = {
    'GTIN' : A_test + A_train,
    'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
    'Actual Level 2' : B_test + B_train,
    'Actuals' : testLevels + B_train,
    'Predictions' : testLevelss + trainLevelss,
    'Scores' : test_new_proobs + train_new_proobs,
    'L1 Name' : level_1
}

df = pd.DataFrame(data)
df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
    'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
    'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
    'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
    'VND_ECOM_DSC', 'Source', 'Actual Level 2', 'Actuals', 'Predictions', 'Scores', 'L1 Name']]

## Keep the output of the below cell for all Level 1 executed

In [ ]:
df.to_csv(DBFO + filenamee + '.csv', index=False)
print(f"{filenamee}.csv   --file created successfully")